In [283]:
import json
import regex
import collections
import numpy as np
import pandas as pd

## Extract ontology events

In [284]:
df = pd.read_excel('KAIROS_Annotation_Tagset_Phase_1_V1.0_argsplit.xlsx', sheet_name='events')

In [279]:
events = {}

for row in df.iterrows():
    row_id, row = row
    event_type = row['Type']
    event_subtype = row['Subtype']
    event_subsubtype = row['Sub-subtype']
    event_definition = row['Definition']
    event_template = row['Template']
    event_args = []
    arg_id = 1
    while True:
        if f'arg{arg_id} label' not in row:
            break
        arg_label = row[f'arg{arg_id} label']
        if pd.isna(arg_label):
            break
        arg_label = arg_label.strip()
        arg_constraints_str = row[f'arg{arg_id} type constraints']
        arg_constraints = arg_constraints_str.strip().split(', ')
        event_args.append((arg_label, arg_constraints))
        arg_id += 1
    if event_subtype == 'Unspecified':
        event_name = event_type
    elif event_subsubtype == 'Unspecified':
        event_name = f'{event_type}.{event_subtype}'
    else:
        event_name = f'{event_type}.{event_subtype}.{event_subsubtype}'
    
    assert event_name not in events
    events[event_name] = {
        'definition': event_definition,
        'template': event_template,
        'args': event_args
    }

In [280]:
def to_short_name(event_name):
    event_type, rest = event_name.split('.', 1)
    event_abbr = ''.join([c for c in event_type if c.isupper()])
    return f'{event_abbr}.{rest}'

## Generate Blockly blocks

In [281]:
def event_to_blocks(event_name, event_body):
    # create top block for a given event
    out_blocks = [{
        'type': f'kairos_event_{event_name}',
        'message0': to_short_name(event_name),
        'message1': '%1',
        'args1': [{
            'type': 'input_statement',
            'name': 'parts'
        }],
        'previousStatement': None,
        'nextStatement': None,
        'style': 'loop_blocks',  # TODO: change
    }]
    # replace <argI> i=1.. with arg names
    # split by |, and create separate blocks for each body part
    event_template = event_body['template']
    body_parts = event_template.split('|')
    for i, body_part in enumerate(body_parts):
        body_part = body_part.strip()
        body_args = []
        body_part_arg_cntr = 0
        for j, (arg_name, _) in enumerate(event_body['args'], 1):
            arg_id_str = f'<arg{j}>'
            if arg_id_str in body_part:
                body_part_arg_cntr += 1
                body_part = body_part.replace(arg_id_str, f'%{body_part_arg_cntr}')
                body_args.append({
                    'type': 'input_value',
                    'name': arg_name,
                    'variable': arg_name
                })
        out_blocks.append({
            'type': f'kairos_event_{event_name}_part_{i}',
            'message0': body_part,
            'args0': body_args,
            'previousStatement': None,
            'nextStatement': None,
            'inputsInline': True,
            'style': 'text_blocks',  # TODO: change
            'extensions': ['kairos_events_checkVarName']
        })
    
    return out_blocks
        

## Generate block JSON descriptions

event_blocks_json = ''

for event_name, event_body in events.items():
    event_blocks = event_to_blocks(event_name, event_body)
    for event_block in event_blocks:
        event_blocks_json += 'Blockly.defineBlocksWithJsonArray([' + json.dumps(event_block) + ']);\n'
    
with open('../kairos_blocks.js', 'w') as fout:
    fout.write(event_blocks_json)

In [288]:
## Generate XML list of blocks

list_of_blocks_xml = '<xml xmlns="https://developers.google.com/blockly/xml" id="toolbox-kairos-blocks" style="display: none">\n'

events_names_dict = collections.defaultdict(list)

for event_name in events.keys():
    event_type = event_name.split('.')[0]
    events_names_dict[event_type].append(event_name)

for event_type, v in events_names_dict.items():
    list_of_blocks_xml += f'<category name="{event_type}" categorystyle="list_category">\n'
    events_names_dict[event_type].sort()
    for event_name in events_names_dict[event_type]:
        list_of_blocks_xml += f'  <block type="kairos_event_{event_name}" editable="false">\n'
        list_of_blocks_xml += f'    <statement name="parts">\n'
        event_args = events[event_name]['args']
        event_body_parts = events[event_name]['template'].split('|')
        for i, body_part in enumerate(event_body_parts):
            list_of_blocks_xml += f'      <block type="kairos_event_{event_name}_part_{i}" movable="false">\n'
            for arg_id_str in regex.findall('<arg(\d+)>', body_part):
                arg_id = int(arg_id_str) - 1
                arg_name = event_args[arg_id][0]
                list_of_blocks_xml += f'        <value name="{arg_name}"><block type="variables_get"><field name="VAR">{arg_name}</field></block></value>\n'
            if i != len(event_body_parts) - 1:
                list_of_blocks_xml += f'      <next>\n'
        for i in range(len(event_body_parts) - 1):
            list_of_blocks_xml += f'      </block>\n'
            list_of_blocks_xml += f'      </next>\n'
        list_of_blocks_xml += f'      </block>\n'
        event_args = events[event_name]['args']
        list_of_blocks_xml += f'    </statement>\n'
        list_of_blocks_xml += '  </block>\n'
    list_of_blocks_xml += '</category>\n'

# Add control blocks
list_of_blocks_xml += '<sep></sep>\n'
list_of_blocks_xml += '<category name="Control blocks" categorystyle="variable_category">\n'
list_of_blocks_xml += '  <block type="kairos_control_parallel"></block>\n'
list_of_blocks_xml += '</category>\n'
list_of_blocks_xml += '</xml>\n'

with open('../kairos_blocks.xml', 'w') as fout:
    fout.write(list_of_blocks_xml)

---